In [1]:
import os
import datetime

import pandas as pd
from steam.webapi import WebAPI
import plotly.express as px

API_KEY = os.environ['API_KEY']

api = WebAPI(API_KEY, raw=False, format='json', https=True, http_timeout=10)



In [2]:
# Create a DataFrame from the extracted data
aj_data = pd.DataFrame([
    {
        'name':'AJ',
        'date': datetime.date.today(),
        'appid':game['appid'],
        'game': game['name'],
        'playtime_forever': game['playtime_forever']/60
    }
    for game in api.call('IPlayerService.GetOwnedGames', include_appinfo=True, appids_filter=None, include_extended_appinfo=False, include_free_sub=False, include_played_free_games=True, steamid=76561198037729598, language='en')['response']['games']
])

# Create a DataFrame from the extracted data
ashton_data = pd.DataFrame([
    {
        'name':'Ashton',
        'date': datetime.date.today(),
        'appid':game['appid'],
        'game': game['name'],
        'playtime_forever': game['playtime_forever']/60
    }
    for game in api.call('IPlayerService.GetOwnedGames', include_appinfo=True, appids_filter=None, include_extended_appinfo=False, include_free_sub=False, include_played_free_games=True, steamid=76561198102561840, language='en')['response']['games']
])

# Create a DataFrame from the extracted data
dickson_data = pd.DataFrame([
    {
        'name':'Dickson',
        'date': datetime.date.today(),
        'appid':game['appid'],
        'game': game['name'],
        'playtime_forever': game['playtime_forever']/60
    }
    for game in api.call('IPlayerService.GetOwnedGames', include_appinfo=True, appids_filter=None, include_extended_appinfo=False, include_free_sub=False, include_played_free_games=True, steamid=76561198044737410, language='en')['response']['games']

])


In [3]:
aj_data = aj_data[aj_data['playtime_forever']>0]
ashton_data = ashton_data[ashton_data['playtime_forever']>0]
dickson_data = dickson_data[dickson_data['playtime_forever']>0]

playtime_by_date_game = pd.concat([aj_data, ashton_data, dickson_data], axis=0)
playtime_by_date_game.head()

,name,date,appid,game,playtime_forever
0,AJ,2023-08-26,70,Half-Life,0.116667
1,AJ,2023-08-26,220,Half-Life 2,0.250000
2,AJ,2023-08-26,340,Half-Life 2: Lost Coast,0.400000
4,AJ,2023-08-26,320,Half-Life 2: Deathmatch,0.133333
5,AJ,2023-08-26,400,Portal,3.300000


In [4]:
playtime_total = playtime_by_date_game.groupby(['name','date']).sum().drop(columns='game').reset_index()
playtime_total.head()

,name,date,appid,playtime_forever
0,AJ,2023-08-26,52790360,2382.600000
1,Ashton,2023-08-26,78884591,2539.316667
2,Dickson,2023-08-26,18781170,2644.366667


In [5]:
game = aj_data[aj_data.playtime_forever == aj_data.playtime_forever.max()]['game'].item()
hours = round(aj_data[aj_data.playtime_forever == aj_data.playtime_forever.max()]['playtime_forever'].item())

print(f'Most played game: {game}, {hours} hours')

Most played game: Apex Legends, 613 hours


In [6]:
unique_game_count = len(aj_data['appid'].unique())

print(f'Unique titles played: {unique_game_count}')

Unique titles played: 123


In [7]:
total_hours = round(aj_data['playtime_forever'].sum())

print(f'Hours played: {total_hours}')

Hours played: 2383


In [12]:
top_10 = aj_data.sort_values('playtime_forever', ascending=True).tail(10)

In [15]:
fig = px.bar(top_10, x='playtime_forever', y='game')
fig.show()